# Ejercicio de tratamiento de datos Clase 2_2

# Instalamos las librerías necesarias e importamos

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml

from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time
import pandas as pd

#1 y 2 Leemos los datos desde NIST y los tratamos

In [ ]:
url = 'https://services.nvd.nist.gov/rest/json/cves/2.0?cpeName=cpe:2.3:o:microsoft:windows_11_21h2:10.0.22000&cvssV3Severity=CRITICAL&resultsPerPage=50&startIndex=0&sourceIdentifier=secure@microsoft.com'
response = requests.get(url)
if response.status_code == 200:
    # Parsear la respuesta JSON
    data = response.json()
    
    # Extraer la lista de vulnerabilidades
    vulnerabilities = data.get('vulnerabilities', [])
    cvssMetrics31 = data.get('cvssMetricV31', [])
    
    # Crear una lista para almacenar los datos
    data_list = []
    
    # Iterar sobre cada vulnerabilidad y extraer la información
    for vuln in vulnerabilities:
        cve_data = vuln.get('cve', {})
        cve_id = cve_data.get('id', '')        
        published = cve_data.get('published', '')
        last_modified = cve_data.get('lastModified', '')
        
        # Extremos la Descripción
        descriptions = cve_data.get('descriptions', [])
        description = descriptions[0].get('value', '') if descriptions else ''
            
        # Extramos datos de Métricas CVSS
        metrics = cve_data.get('metrics', {})
                
        # Extraemos datos de CVSS v3.1
        cvss_v31 = metrics.get('cvssMetricV31', [])
        if cvss_v31:
            cvss_v31_data = cvss_v31[0].get('cvssData', {})
            cvss_v31_score = cvss_v31_data.get('baseScore', '')
            cvss_v31_vector = cvss_v31_data.get('vectorString', '')
            attackComplexity = cvss_v31_data.get('attackComplexity', '')
            privilegesRequired = cvss_v31_data.get('privilegesRequired', '')
            userInteraction = cvss_v31_data.get('userInteraction', '')
            scope = cvss_v31_data.get('scope', '')
            confidentiality = cvss_v31_data.get('confidentialityImpact', '')
            integrity = cvss_v31_data.get('integrityImpact', '')
            availability = cvss_v31_data.get('availabilityImpact', '')            
                        
        else:
            cvss_v31_score = ''
            cvss_v31_vector = ''
        
        # Añadimos los datos extraídos a una lista
        data_list.append({
            'CVE_ID': cve_id,
            'Published': published,
            'LastModified': last_modified,
            'Description': description,
            'CVSSv3.1_Score': float(cvss_v31_score),
            'CVSSv3.1_Vector': cvss_v31_vector,
            'attackComplexity': attackComplexity,
            'privilegesRequired': privilegesRequired,
            'userInteraction': userInteraction,
            'scope': scope,
            'confidentialityImpact': confidentiality,
            'integrityImpact': integrity,
            'availabilityImpact': availability            
        })
    
    # Creamos un DataFrame con pandas
    df = pd.DataFrame(data_list)
    
    # Seleccionamos las primeras 10 filas y 13 columnas
    src = df.iloc[:10, :13]

    #Verificamos si hay valores nulos
    print(df.isnull().sum())
    
    #Comprobamos el tipo de datos de la columna CVE_ID
    tipo_dato = df['CVE_ID'].dtype
    print(f"El tipo de dato de la columna es: {tipo_dato}")
    tipos_elementos = df['CVE_ID'].apply(type).value_counts()
    print("Tipos de datos en la columna 'CVE_ID':")
    print(tipos_elementos)
    
    # Si queremos mostrar los datos extraídos de NIST descomentar la siguiente línea
    #print(src.head())
else:
    print(f"Error: {response.status_code}")
    print(response.text)

#Una vez que creamos la BDD "vulnerabilidades" y la tabla "nist_cves", guardamos los datos del data frame a la tabla nist_cves

In [ ]:
#******* Llenar datos en BDD vulnerabilidades y tabla NIst_CVEs
import psycopg2
import json
from datetime import datetime

# Conectamos a la base de datos vulnerabilidades 
conn = psycopg2.connect(
    database="vulnerabilidades",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()
       
# Insertar registros en la tabla
insert_query = """
INSERT INTO NIST_CVEs
(
    cve_id,
    Published,
    LastModified,
    Description,
    cvss_v31_score,
    attackVector,
    attackComplexity,
    privilegesRequired,
    userInteraction,
    scope,
    confidentialityImpact,
    integrityImpact,
    availabilityImpact
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
# Preparar los datos como una lista de tuplas
data_to_insert = [tuple(row) for index, row in src.iterrows()]

# Ejecutar insert de múltiples filas a la vez usando executemany
cursor.executemany(insert_query, data_to_insert)

# Confirmar la transacción
conn.commit()
# Cerrar el cursor y la conexión
cursor.close()
conn.close()

print("¡Datos insertados correctamente!")



#Ejecutamos un SELECT a la tabla nist_cves para verificar que los datos se han guardado correctamente

In [ ]:
# Conectamos a la base de datos vulnerabilidades
conn = psycopg2.connect(
    database="vulnerabilidades",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()
cursor.execute("SELECT * FROM nist_cves;")

# Obtener todos los resultados de la consulta
rows = cursor.fetchall()

# Mostramos los resultados como un DataFrame de pandas (opcional)
columns = [desc[0] for desc in cursor.description]  # Extraemos los nombres de las columnas
df = pd.DataFrame(rows, columns=columns)

# Cerramos el cursor y la conexión
cursor.close()
conn.close()

# Mostramos el DataFrame con los resultados
df